In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 28 15:34:42 2020

Purpose: AN6001 Project

@author: vivek
"""

import os

os.chdir("C:/Users/vivek/Documents/MSBA/AN6001 Big Data and AI in Analytics/Group Project/nlp-getting-started")

print(os.getcwd())

#%%

import pandas as pd

TrainData = pd.read_csv("train.csv")
TestData = pd.read_csv("test.csv")
SampleSubmitssionData = pd.read_csv("sample_submission.csv")

print(TrainData.head())
print(TestData.head())
print(SampleSubmitssionData.head())
#%%
import numpy as np
print(TrainData)
print(TestData)
print(SampleSubmitssionData)

#%%

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords

stop=set(stopwords.words('english'))

print(stop)

#%%

def create_corpus(target):
    corpus=[]
    
    for x in TrainData[TrainData['target']==target]['text'].str.split():
        for i in x:
            corpus.append(i)
    return corpus

corpus=create_corpus(0)

#%%
import string
special = string.punctuation

#%%
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

dic=defaultdict(int)
for i in (corpus):
    if i in special:
        dic[i]+=1

x,y=zip(*dic.items())
plt.bar(x,y)

#%%

## Data Cleaning

df=pd.concat([TrainData,TestData])
df.shape


#%%

##Removing Urls

import re
example="New competition launched :https://www.kaggle.com/c/nlp-getting-started"
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub('',text)

print(remove_URL(example))

df['text']=df['text'].apply(lambda x : remove_URL(x))

#%%

## Removing HTML Tags

example = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

print(remove_html(example))

df['text']=df['text'].apply(lambda x: remove_html(x))

#%%

## Removing Punctuations

def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)

example="I am a #king"
print(remove_punct(example))

df['text']=df['text'].apply(lambda x : remove_punct(x))


#%%

## Spelling Correction

from spellchecker import SpellChecker
from tqdm import tqdm

spell = SpellChecker()

# Spell Correction, Lemmatization and Create corpus of Words

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

import json

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def create_corpus(df):
    wordnet_lemmatizer = WordNetLemmatizer()
    corpus=[]
    for tweet in tqdm(df['text']):
        #print(tweet)
        misspelled_words = spell.unknown(tweet.split())
        corrected_text = [spell.correction(word) if word in misspelled_words else word for word in word_tokenize(tweet.lower())]
        #print(corrected_text)
        #tagged = nltk.pos_tag(corrected_text)
        #wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), tagged)
        #lemt_word = [word if tag is None else wordnet_lemmatizer.lemmatize(word, tag) for word, tag in wordnet_tagged]
        #print(lemt_word)
        words = [word for word in corrected_text if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
        #print(words)
    return corpus

corpus=create_corpus(df)

with open('corpus1.txt', 'w') as filehandle:
    json.dump(corpus, filehandle)
    
#%%

import json
test = [["I", "am", "saving", "text", "array"],["is", "it", "saved"]]


with open('output.txt', 'w') as filehandle:
    json.dump(test, filehandle)

check = []
with open('corpus1.txt', 'r') as filehandle:
    check = json.load(filehandle)

corpus = check
#print(check)

#%%

# Creating an embedded dictionary from Glove Txt file of 100 dimensions
from tqdm import tqdm
import numpy as np
embedding_dict={}
with open('glove.6B/glove.6B.100d.txt','r', encoding="utf8") as f:
    for line in tqdm(f):
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

#%%

# Creating Sequences of words from corpus

from keras.preprocessing.text import Tokenizer

MAX_LEN=50
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

#%%

# Padding and storing Word Indexes

from keras.preprocessing.sequence import pad_sequences

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

#%%

# Creating Embdedding Matrix from Word Index and Embedding Dictionary 
# created from Globe Text

num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,100))

for word,i in tqdm(word_index.items()):
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec
        
#%%

from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.optimizers import Adam
from keras.initializers import Constant

model=Sequential()

embedding=Embedding(num_words,100,
                    embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding)

model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

optimzer=Adam(learning_rate=1e-5)

model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['accuracy'])

#%%

# Get the Trains and Test set from combined rows

train=tweet_pad[:TrainData.shape[0]]
test=tweet_pad[TrainData.shape[0]:]

#train=df['text'][:TrainData.shape[0]]
#test=df['text'][TrainData.shape[0]:]

#%%

# Divide the Train Data in Train and Validation Set

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(train,
                                               TrainData['target'].values,
                                               test_size=0.15)
print('Shape of train',X_train.shape)
print("Shape of Validation ",X_test.shape)

#%%

# Train the model

history=model.fit(X_train,y_train,batch_size=4,epochs=15,validation_data=(X_test,y_test),verbose=2)

#%%

y_pred = model.predict(X_test, batch_size=64, verbose=1)
#%%

y_pred_bool = []
for i in y_pred:
    if i>0.5:
        y_pred_bool.append(1)
    else:
        y_pred_bool.append(0)
        
#%%
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_train_pred = model.predict(X_train, batch_size=64, verbose=1)
#%%

from sklearn.metrics import confusion_matrix
y_pred_bool = []
for i in y_pred:
    if i>0.5:
        y_pred_bool.append(1)
    else:
        y_pred_bool.append(0)

y_train_pred_bool = []
for i in y_train_pred:
    if i>0.5:
        y_train_pred_bool.append(1)
    else:
        y_train_pred_bool.append(0)
#%%

